# TP3 : Word2vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

### Chargement et traitement des phrases du corpus

In [2]:
# Création d'un objet qui streame les lignes d'un fichier pour économiser de la RAM

class MySentences(object):
    """Tokenize and Lemmatize sentences""" #permet de tokenizer des "doubles" mots, par ex "premier" + "ministre" va être considéré comme un token "premier ministre"
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

infile = f"../data/sents.txt"
sentences = MySentences(infile)

#### Détection des bigrams

In [ ]:
# Création de l'objet 'phrases' = "dictionnaire d'expressions multi-mots associées à un score", dont les clés correspondent aux termes du corpus

bigram_phrases = Phrases(sentences)

In [ ]:
# Conversion des objets 'phrases' en objet 'phraser' = version light du 'phrases' -> convertit certains unigrams en bigrams s'ils sont pertinents

bigram_phraser = Phraser(phrases_model=bigram_phrases)

#### Détection des trigrams

In [ ]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

#### Créations d'un corpus d'unigrams, bigrams, trigrams

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

print(corpus[:100])

#imprime une liste de n-grammes, qu'on répère car ils sont séparés par des _

#cf. library "pickle" pour sauvegarder objet python en format binaire

### Entraînement d'un modèle word2vec sur ce corpus

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=1, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=10, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=2, # Permet de paralléliser l'entraînement du modèle en 4 threads -> si la machine est sur le point d'exploser, diminuer
    epochs=2 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.

# pour tester: petite fenêtre de mots, peu de workers, haut min_count, peu d'epoch -> puis améliorer itérativement le modèle

)

# Sauver le modèle dans un fichier
outfile = f"../data/newspapers_window1_mincount10_epochs2.model"
model.save(outfile)

### Exploration du modèle

In [ ]:
# Charger le modèle en mémoire

model = Word2Vec.load("../data/newspapers.model")

In [ ]:
# Imprimer le vecteur d'un terme

model.wv["ministre"] #si erreur, vérifier si le mot apparaît bien dans la sélection (surtout si on garde juste des mots très fréquents)

In [ ]:
# Calculer la similarité entre deux termes

model.wv.similarity("ministre", "roi")

In [ ]:
# Chercher les mots les plus proches d'un terme donné

model.wv.most_similar("ministre", topn=10)

In [ ]:
# Faire des recherches complexes à travers l'espace vectoriel

print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique'])) #positive = proche, negative=éloigné